In [1]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 48.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.7 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0

In [2]:
import cv2 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import time 
import mediapipe as mp 
import torch 
import torch.nn as nn 
from tqdm import tqdm


2025-05-04 14:24:45.715504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746368685.958624      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746368686.025721      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils  

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

##1.Data Preprocessing

In [5]:
def normalize_keypoint_block(block,dim = 3):
    if np.all(block == 0):
        return block.flatten()

    block = block.reshape(-1,dim)
    mean = np.mean(block,axis = 0)
    std = np.std(block,axis = 0) + 1e-6
    normed = (block - mean)/std 
    return normed.flatten()

def normalize_frames(frame):
    pose = frame[:132].reshape(33,4)
    left = frame[132:195].reshape(21,3)
    right = frame[195:258].reshape(21,3)

    pose_norm = normalize_keypoint_block(pose,dim = 4)
    #face_norm = normalize_keypoint_block(face,dim = 3)
    left_norm = normalize_keypoint_block(left,dim = 3)
    right_norm = normalize_keypoint_block(right,dim = 3)

    return np.concatenate([pose_norm,left_norm,right_norm])

def normalize_keypoints(keypoints):
    return np.array([normalize_frames(frame) for frame in keypoints])

In [6]:
tmp = np.load('/kaggle/input/dataset/cc/hello/Recording from 2025-05-02 14-24-31.714738.npy')
print(tmp)
print(tmp.shape)

[[ 0.50361878  0.5505414  -0.67453498 ...  0.          0.
   0.        ]
 [ 0.50396407  0.55145919 -0.56418455 ...  0.          0.
   0.        ]
 [ 0.50490034  0.5521304  -0.5815779  ...  0.          0.
   0.        ]
 ...
 [ 0.50747389  0.5675236  -0.47857314 ...  0.30349231  0.24728128
  -0.01748259]
 [ 0.5071882   0.56733674 -0.47297597 ...  0.30498332  0.24849832
  -0.0193427 ]
 [ 0.50698316  0.56729734 -0.47312039 ...  0.30592594  0.24741414
  -0.01826371]]
(29, 258)


In [7]:
!ls -r "/kaggle/input/dataset/cc"

you  yes  sit  please  nice  name  my  meet  hello


In [8]:
from sklearn.model_selection import train_test_split
from shutil import copy2

In [9]:
file_paths = []
labels = []
data_dir = '/kaggle/input/dataset/cc'
DEST_DIR = '/kaggle/working/split_data'

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir,class_name)
    if not os.path.isdir(class_dir):
        continue 
    for fname in os.listdir(class_dir):
        if fname.endswith('.npy'):
            file_paths.append(os.path.join(class_dir, fname))
            labels.append(class_name)

train_files, valid_files, train_labels, valid_labels = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)
def copy_split(files, labels, split_name):
    for fpath, label in tqdm(zip(files, labels), total=len(files), desc=split_name):
        out_dir = os.path.join(DEST_DIR, split_name, label)
        os.makedirs(out_dir, exist_ok=True)
        copy2(fpath, os.path.join(out_dir, os.path.basename(fpath)))

copy_split(train_files, train_labels, 'train')
copy_split(valid_files, valid_labels, 'valid')

valid: 100%|██████████| 101/101 [00:00<00:00, 159.12it/s]


In [11]:
data_path = '/kaggle/working/split_data/train'
actions = os.listdir(data_path)
for action in tqdm(actions,desc = "Action"):
    action_path = os.path.join(data_path,action)
    for file_name in os.listdir(action_path):
        file_path = os.path.join(action_path,file_name)
        keypoints = np.load(file_path)
        if keypoints.shape[1] != 258:
            continue
        normed = normalize_keypoints(keypoints)
        np.save(file_path,normed)

Action: 100%|██████████| 9/9 [00:03<00:00,  2.96it/s]


In [12]:
data_path = '/kaggle/working/split_data/valid'
actions = os.listdir(data_path)
for action in tqdm(actions,desc = "Action"):
    action_path = os.path.join(data_path,action)
    for file_name in os.listdir(action_path):
        file_path = os.path.join(action_path,file_name)
        keypoints = np.load(file_path)
        if keypoints.shape[1] != 258:
            continue
        normed = normalize_keypoints(keypoints)
        np.save(file_path,normed)

Action: 100%|██████████| 9/9 [00:00<00:00, 11.58it/s]


In [13]:
path = '/kaggle/working/split_data/train'
allowed_action = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
len(allowed_action)

9

In [14]:
tmp = '/kaggle/working/split_data/valid'
tmp_actions = [name for name in os.listdir(tmp) if os.path.isdir(os.path.join(tmp, name))]
len(tmp_actions)

9

In [15]:
tmp = np.load('/kaggle/working/split_data/train/hello/Recording from 2025-05-02 14-24-31.714738.npy')
print(tmp)
print(tmp.shape)

[[ 0.05929839 -1.04665421 -0.83452906 ...  0.          0.
   0.        ]
 [ 0.00872303 -1.06056208 -0.91760498 ...  0.          0.
   0.        ]
 [-0.00492156 -1.06354973 -0.96056232 ...  0.          0.
   0.        ]
 ...
 [ 0.04614017 -0.80394014 -0.65004444 ... -0.67396612 -0.81890504
  -1.60890735]
 [ 0.04353278 -0.80461785 -0.65625849 ... -0.64550359 -0.81426503
  -1.66546222]
 [ 0.04127396 -0.80469538 -0.64753239 ... -0.65709426 -0.8319299
  -1.6576056 ]]
(29, 258)


In [16]:
label_map = {label:num for num,label in enumerate(allowed_action)}
label_map

{'name': 0,
 'you': 1,
 'my': 2,
 'please': 3,
 'sit': 4,
 'meet': 5,
 'nice': 6,
 'yes': 7,
 'hello': 8}

In [17]:
sequences_train = []
labels_train = []
DATA_PATH = '/kaggle/working/split_data/train'
expected_len = 80
for action in sorted(allowed_action):
    action_path = os.path.join(DATA_PATH,action)
    if not os.path.exists(action_path):
        continue
        
    for file_name in os.listdir(action_path):
        file_path = os.path.join(action_path,file_name)
        sequence = np.load(file_path)

        if sequence.shape[0] < expected_len:
            pad_len = expected_len - sequence.shape[0]
            pad = np.zeros((pad_len,sequence.shape[1]))
            sequence = np.concatenate((sequence,pad),axis = 0)

        if sequence.shape[0] > expected_len:
            start = (sequence.shape[0] - expected_len)//2
            sequence = sequence[start:start + expected_len]

        if sequence.shape == (expected_len,258): 
            sequences_train.append(sequence)
            labels_train.append(label_map[action])
    

In [19]:
sequences_valid = []
labels_valid = []
DATA_PATH = '/kaggle/working/split_data/valid'
expected_len = 80
for action in sorted(allowed_action):
    action_path = os.path.join(DATA_PATH,action)
    if not os.path.exists(action_path):
        continue
        
    for file_name in os.listdir(action_path):
        file_path = os.path.join(action_path,file_name)
        sequence = np.load(file_path)

        if sequence.shape[0] < expected_len:
            pad_len = expected_len - sequence.shape[0]
            pad = np.zeros((pad_len,sequence.shape[1]))
            sequence = np.concatenate((sequence,pad),axis = 0)

        if sequence.shape[0] > expected_len:
            start = (sequence.shape[0] - expected_len)//2
            sequence = sequence[start:start + expected_len]

        if sequence.shape == (expected_len,258): 
            sequences_valid.append(sequence)
            labels_valid.append(label_map[action])
    

In [21]:
len(sequences_train)

403

In [22]:
len(labels_train)

403

In [52]:
from collections import Counter
print(Counter(labels_train))


Counter({8: 51, 5: 44, 2: 44, 0: 44, 6: 44, 3: 44, 4: 44, 7: 44, 1: 44})


In [23]:
len(sequences_valid)


101

In [24]:
len(labels_valid)

101

In [26]:
X_train = np.array(sequences_train)
y_train = np.array(labels_train)

In [27]:
print(X_train[0])
print(X_train[0].shape)

[[ 0.03536706 -1.06771555 -0.85784163 ...  0.          0.
   0.        ]
 [ 0.12038135 -1.03571908 -0.72858462 ...  0.          0.
   0.        ]
 [ 0.0784345  -0.92066243 -0.50146382 ...  0.62742589 -0.55823721
  -1.13890556]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
(80, 258)


In [28]:
X_valid = np.array(sequences_valid)
y_valid = np.array(labels_valid)

In [29]:
print(X_valid[0])
print(X_valid[0].shape)

[[-0.0157693  -1.0543814  -0.96317528 ...  0.          0.
   0.        ]
 [-0.01016033 -1.05697173 -0.93759595 ...  0.          0.
   0.        ]
 [ 0.0099618  -1.05828659 -0.91806101 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
(80, 258)


In [31]:
import torch.nn as nn 
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]
    

In [32]:
from torch.utils.data import DataLoader
keypoint_dataset_train = CustomDataset(X_train,y_train)
keypoint_dataset_val = CustomDataset(X_valid,y_valid)
train_loader = DataLoader(keypoint_dataset_train,batch_size = 32,shuffle = True)
valid_loader = DataLoader(keypoint_dataset_val, batch_size = 32, shuffle = False )

In [34]:
len(train_loader)

13

In [35]:
len(valid_loader)

4

##2.Build Model

In [73]:
class LSTMModel(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(LSTMModel,self).__init__()
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers = 1,batch_first = True,bidirectional = True)
        self.dropout = nn.Dropout(0.6)
        self.fc1 = nn.Linear(hidden_size*2,hidden_size)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,output_size)
    
    def forward(self,x):
        x, _ = self.lstm(x)
        x = torch.max(x,dim = 1).values
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x 

In [74]:
input_size = 258
hidden_size = 160
output_size = len(allowed_action)
model = LSTMModel(input_size,hidden_size,output_size)

In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [76]:
model = model.to(device)
print(model)

LSTMModel(
  (lstm): LSTM(258, 160, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.6, inplace=False)
  (fc1): Linear(in_features=320, out_features=160, bias=True)
  (bn): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=160, out_features=9, bias=True)
)


In [77]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr = 1e-3,weight_decay = 1e-5)
loss_function = nn.CrossEntropyLoss()

In [78]:
def train():
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x,y in train_loader:
        x = x.float().to(device)
        y = y.long().to(device)
        output = model(x)
        optimizer.zero_grad()
        loss = loss_function(output,y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(output,1)
        correct += (predicted == y).sum().item()
        total += y.size(0)
    accuracy = (100*correct)/total
    avg_loss = running_loss/total
    print(f'Loss:{avg_loss: .4f}  Accuracy: {accuracy: .2f}%')
        

In [79]:
def valid():
    running_loss = 0
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x,y in valid_loader:
            x = x.float().to(device)
            y = y.long().to(device)
            output = model(x)
            loss = loss_function(output,y)
            running_loss += loss.item() * y.size(0)
            _, predicted = torch.max(output,1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    avg_loss = running_loss / total
    accuracy = (correct*100)/total
    print(f'Loss:{avg_loss: .4f}  Accuracy:{accuracy: .2f}%')

In [80]:
num_epochs = 100
for epoch in range(num_epochs):
    print(f'Epoch: {epoch+1}/{num_epochs}:')
    train()
    valid()

Epoch: 1/100:
Loss: 0.0510  Accuracy:  57.07%
Loss: 1.3755  Accuracy: 100.00%
Epoch: 2/100:
Loss: 0.0199  Accuracy:  98.26%
Loss: 0.4432  Accuracy: 100.00%
Epoch: 3/100:
Loss: 0.0092  Accuracy:  100.00%
Loss: 0.1488  Accuracy: 100.00%
Epoch: 4/100:
Loss: 0.0054  Accuracy:  99.75%
Loss: 0.0655  Accuracy: 100.00%
Epoch: 5/100:
Loss: 0.0033  Accuracy:  100.00%
Loss: 0.0373  Accuracy: 100.00%
Epoch: 6/100:
Loss: 0.0024  Accuracy:  99.75%
Loss: 0.0223  Accuracy: 100.00%
Epoch: 7/100:
Loss: 0.0018  Accuracy:  100.00%
Loss: 0.0199  Accuracy: 100.00%
Epoch: 8/100:
Loss: 0.0014  Accuracy:  100.00%
Loss: 0.0138  Accuracy: 100.00%
Epoch: 9/100:
Loss: 0.0010  Accuracy:  100.00%
Loss: 0.0108  Accuracy: 100.00%
Epoch: 10/100:
Loss: 0.0009  Accuracy:  100.00%
Loss: 0.0086  Accuracy: 100.00%
Epoch: 11/100:
Loss: 0.0008  Accuracy:  100.00%
Loss: 0.0074  Accuracy: 100.00%
Epoch: 12/100:
Loss: 0.0006  Accuracy:  100.00%
Loss: 0.0065  Accuracy: 100.00%
Epoch: 13/100:
Loss: 0.0006  Accuracy:  100.00%
Loss:

In [55]:
print(device)

cuda


In [81]:
classes = ['hello','meet','my','name','nice','please','sit','yes','you']

# Test 1 file npy
x = np.load("/kaggle/working/split_data/valid/sit/Recording from 2025-05-02 18-00-29.587158.npy")  # sửa lại đường dẫn
x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)

model = model.to(device)
with torch.no_grad():
    output = model(x_tensor)
    probs = torch.softmax(output, dim=1)[0].cpu().numpy()
    pred = classes[np.argmax(probs)]

print("Probability:", pred)
print("Probability of each classes", probs)

Probability: nice
Probability of each classes [5.1114348e-06 4.2053339e-06 6.1613118e-06 3.6238623e-06 9.9996102e-01
 6.0527527e-06 6.4791070e-06 3.8693638e-06 3.4992017e-06]


In [82]:
torch.save(model.state_dict(),'best_model.pt')